In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.svm import SVC
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import seaborn as sns
import warnings
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore")
simplefilter("ignore", category=ConvergenceWarning)
%matplotlib inline

In [47]:
df_1 = pd.read_csv('indicators.csv', parse_dates= True, index_col=0)
df_2 = pd.read_csv('../Data/BTC.csv', parse_dates= True, index_col=0)
df = df_1.merge(df_2, left_index=True, right_index=True, how='inner')
df['close'] = df['close_x']
df

,close_x,Chaikin_oscillator,CCI,williams_percentage,PSAR,Keltner_channels_upperband,Keltner_channels_lowerband,TRIX,Ultimate_oscillator,ad_indicator,...,bollinger_band_upper,ADX,return,target,open,high,low,close_y,volume,close
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-03-01 00:00:00+00:00,10929.37,-5.792097e+07,74.580435,-9.587841,10142.146888,11190.338986,9464.105603,0.226935,0.535553,1.005499e+10,...,11802.987291,32.765792,2.917708,1,10334.44,11090.30,10247.56,10929.37,9.524721e+08,10929.37
2018-03-02 00:00:00+00:00,11043.12,-1.120244e+07,93.556571,-14.341332,10236.962199,11234.570293,9680.201991,0.275395,0.523143,1.025779e+10,...,11855.073963,31.744213,2.545677,1,10929.37,11191.94,10801.45,11043.12,8.529085e+08,11043.12
2018-03-03 00:00:00+00:00,11465.36,1.121147e+08,118.798268,-5.089237,10351.559535,11365.176210,9916.133470,0.338236,0.533494,1.085243e+10,...,11842.217069,31.114049,2.865705,1,11043.12,11530.66,11043.12,11465.36,8.122001e+08,11465.36
2018-03-04 00:00:00+00:00,11504.42,1.913912e+08,113.967555,-2.737129,10516.633600,11473.362703,10122.044414,0.405838,0.569436,1.143595e+10,...,11922.623598,30.795111,1.471089,1,11465.36,11539.79,11084.01,11504.42,6.907273e+08,11504.42
2018-03-05 00:00:00+00:00,11440.73,-6.621836e+07,126.973939,-28.388036,10680.338624,11515.175050,10314.060483,0.468626,0.536724,1.069981e+10,...,11854.363273,30.827109,-0.256666,0,11503.94,11694.15,11431.55,11440.73,7.914719e+08,11440.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31 00:00:00+00:00,28477.29,1.202877e+08,69.580631,-27.683656,26711.787977,28937.672239,26436.050918,0.798336,0.556204,1.274664e+11,...,30348.323537,41.991473,0.661301,0,28037.46,28650.47,27541.23,28477.29,1.436740e+09,28477.29
2023-04-01 00:00:00+00:00,28465.30,2.131430e+07,88.350836,-37.024828,26828.108898,28950.973650,26705.818023,0.782692,0.556223,1.273159e+11,...,29955.810060,41.877595,0.489350,0,28477.29,28810.95,28265.42,28465.30,5.632994e+08,28465.30
2023-04-02 00:00:00+00:00,28186.76,-2.040043e+07,68.205514,-60.437466,26986.736186,28931.554805,26855.551110,0.761019,0.541949,1.272746e+11,...,29814.813232,41.486941,-0.668477,0,28465.30,28538.36,27880.95,28186.76,5.925811e+08,28186.76


In [48]:
df = df[['open','high','low','close','volume','Chaikin_oscillator', 'williams_percentage', 'Stochastic_oscillator','return','target']]
df

,open,high,low,close,volume,Chaikin_oscillator,williams_percentage,Stochastic_oscillator,return,target
timestamp,,,,,,,,,,
2018-03-01 00:00:00+00:00,10334.44,11090.30,10247.56,10929.37,9.524721e+08,-5.792097e+07,-9.587841,90.860716,2.917708,1
2018-03-02 00:00:00+00:00,10929.37,11191.94,10801.45,11043.12,8.529085e+08,-1.120244e+07,-14.341332,91.639889,2.545677,1
2018-03-03 00:00:00+00:00,11043.12,11530.66,11043.12,11465.36,8.122001e+08,1.121147e+08,-5.089237,95.255809,2.865705,1
2018-03-04 00:00:00+00:00,11465.36,11539.79,11084.01,11504.42,6.907273e+08,1.913912e+08,-2.737129,97.262871,1.471089,1
2018-03-05 00:00:00+00:00,11503.94,11694.15,11431.55,11440.73,7.914719e+08,-6.621836e+07,-28.388036,82.481560,-0.256666,0
...,...,...,...,...,...,...,...,...,...,...
2023-03-31 00:00:00+00:00,28037.46,28650.47,27541.23,28477.29,1.436740e+09,1.202877e+08,-27.683656,73.319142,0.661301,0
2023-04-01 00:00:00+00:00,28477.29,28810.95,28265.42,28465.30,5.632994e+08,2.131430e+07,-37.024828,71.839170,0.489350,0
2023-04-02 00:00:00+00:00,28465.30,28538.36,27880.95,28186.76,5.925811e+08,-2.040043e+07,-60.437466,48.400701,-0.668477,0


In [49]:
df.isnull().sum()

open                     0
high                     0
low                      0
close                    0
volume                   0
Chaikin_oscillator       0
williams_percentage      0
Stochastic_oscillator    0
return                   0
target                   0
dtype: int64

In [50]:
#t = '2021-09-06'
t = int(0.8 * len(df))
X = df.drop(['target'], axis=1)
y = df.target
X_train = X.iloc[:t]
y_train = y.iloc[:t]
X_test = X.iloc[t:]
y_test = y.iloc[t:]
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()

In [51]:
X_train.shape

(1488, 9)

In [52]:
X_test.shape

(373, 9)

In [53]:
model_params = {
    'svm': {
        'model': svm.SVC(),
        'params' : {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf', 'poly'],
             'gamma': ['scale', 'auto'],
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [10, 50, 100, 200],
            # 'max_depth': [3, 5, 7, 9],
            # 'min_samples_split': [2, 5, 10],
            # 'min_samples_leaf': [1, 2, 4],
            # 'max_features': ['sqrt', 'log2']
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
               'C': [0.1, 1, 10],
                # 'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                # 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                # 'max_iter': [100, 500, 1000],
          
        }
    },
     'naive_bayes_gaussian': {
         'model': GaussianNB(),
         'params': {
              'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5],
         }
     },
    
     'decision_tree': {
         'model': DecisionTreeClassifier(),
         'params': {
             'max_depth': [None, 5, 10, 15, 20],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini','entropy'],
            
         }
     },
     'gradient_boost':{
         'model': GradientBoostingClassifier(),
         'params':{
         "learning_rate": [0.001, 0.01, 0.1, 1],
         "n_estimators": [50, 100, 200, 300, 400, 500],
         "max_depth": [2, 3, 4, 5],
         }
     },     
     'adaboost':{
         'model': AdaBoostClassifier(),
         'params':{
         "learning_rate": [0.001, 0.01, 0.1, 1],
         'n_estimators': [50, 100, 200, 300, 400, 500],
         'algorithm': ['SAMME', 'SAMME.R']
         }
     }    
}


In [54]:
window = 365
n_splits = (df.iloc[:t].shape[0] - window) // window
tscv = TimeSeriesSplit(n_splits=n_splits)
tscv

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)

In [55]:
scores = []
best_estimators = {}
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=tscv, return_train_score=False,n_jobs=-1)
    clf.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    best_estimators[model_name] = clf.best_estimator_
    
result = pd.DataFrame(scores,columns=['model','best_score','best_params'])
result.sort_values(by='best_score', ascending=False)

,model,best_score,best_params
4,decision_tree,0.997312,"{'criterion': 'entropy', 'max_depth': 20, 'max..."
5,gradient_boost,0.996416,"{'learning_rate': 0.001, 'max_depth': 2, 'n_es..."
6,adaboost,0.996416,"{'algorithm': 'SAMME', 'learning_rate': 0.01, ..."
1,random_forest,0.994624,{'n_estimators': 200}
0,svm,0.976703,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}"
2,logistic_regression,0.906810,{'C': 10}
3,naive_bayes_gaussian,0.737455,{'var_smoothing': 1e-09}


In [56]:
results = pd.DataFrame()
features = {}
results_list = []
for k, v in best_estimators.items():
    model = best_estimators[k]
    model.score(X_test, y_test)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    results_list.append({'Model': k, 'Accuracy': accuracy, 
                          'Precision': precision, 
                          'Recall': recall,
                          'F1': f1
                         })

results = pd.concat([pd.DataFrame(res, index=[0]) for res in results_list], ignore_index=True)
results.sort_values(by='Accuracy', ascending=False, inplace=True)
results

,Model,Accuracy,Precision,Recall,F1
4,decision_tree,0.924933,0.934510,0.924933,0.925269
5,gradient_boost,0.924933,0.934510,0.924933,0.925269
6,adaboost,0.924933,0.934510,0.924933,0.925269
1,random_forest,0.922252,0.932601,0.922252,0.922596
0,svm,0.903485,0.918509,0.903485,0.903816
2,logistic_regression,0.884718,0.894437,0.884718,0.884477
3,naive_bayes_gaussian,0.836461,0.846774,0.836461,0.835098
